In [ ]:
# Install required packages
!pip install streamlit scikit-learn plotly stqdm pyngrok
!pip install numpy pandas scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import numpy as np
import pandas as pd
import streamlit as st
from scipy.constants import g
from scipy.optimize import minimize
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import plotly.express as px
from stqdm import stqdm

# Enhanced configuration and constants
MATERIAL_DB = {
    'aluminum_2024': {'density': 2780, 'youngs': 73.1e9, 'max_stress': 324e6, 'cost': 3.5, 'thermal_conductivity': 121},
    'titanium_6Al-4V': {'density': 4430, 'youngs': 113.8e9, 'max_stress': 950e6, 'cost': 15.2, 'thermal_conductivity': 7.2},
    'carbon_composite': {'density': 1600, 'youngs': 70e9, 'max_stress': 600e6, 'cost': 45.0, 'thermal_conductivity': 5.0},
    'inconel_718': {'density': 8190, 'youngs': 200e9, 'max_stress': 1200e6, 'cost': 28.5, 'thermal_conductivity': 11.4},
    'maraging_steel': {'density': 8000, 'youngs': 190e9, 'max_stress': 2000e6, 'cost': 12.7, 'thermal_conductivity': 20}
}

MIN_SAFETY_FACTOR = 1.5
MAX_MASS_PER_STAGE = 50000
MAX_Q = 50e3  # Max dynamic pressure (Pa)
MAX_G_LOAD = 5.0  # Max acceleration (g)

class LaunchVehicleDesigner:
    def __init__(self):
        self.surrogate_model = None
        self.design_history = []
        self.performance_history = []

    def generate_initial_population(self, n_designs):
        """Generate initial population of designs with more sophisticated parameters"""
        designs = []
        for _ in range(n_designs):
            design = {
                'diameter': np.random.uniform(1.0, 5.0),
                'length': np.random.uniform(10.0, 100.0),
                'wall_thickness': np.random.uniform(0.005, 0.1),
                'material': np.random.choice(list(MATERIAL_DB.keys())),
                'payload_mass': np.random.uniform(1000, 20000),
                'stage_count': np.random.randint(1, 4),
                'nosecone_ratio': np.random.uniform(0.5, 3.0),
                'fin_count': np.random.randint(3, 6),
                'fin_span': np.random.uniform(0.5, 2.0),
                'engine_type': np.random.choice(['liquid', 'solid', 'hybrid']),
                'propellant_mass_ratio': np.random.uniform(0.7, 0.95)
            }

            # Generate stage-specific parameters
            for stage in range(design['stage_count']):
                design[f'stage_{stage}_length_ratio'] = np.random.uniform(0.2, 0.8)
                design[f'stage_{stage}_diameter_ratio'] = np.random.uniform(0.8, 1.2)
                design[f'stage_{stage}_thickness_ratio'] = np.random.uniform(0.8, 1.2)

            designs.append(design)
        return designs

    def calculate_performance(self, design):
        """Enhanced performance calculation with more metrics"""
        mat_props = MATERIAL_DB[design['material']]
        diameter = design['diameter']
        length = design['length']
        thickness = design['wall_thickness']
        density = mat_props['density']
        youngs = mat_props['youngs']
        payload = design['payload_mass']
        stages = design['stage_count']

        # Calculate stage-specific parameters
        stage_lengths = []
        stage_masses = []
        total_length = 0

        for stage in range(stages):
            stage_length = length * design.get(f'stage_{stage}_length_ratio', 1.0/stages)
            stage_diameter = diameter * design.get(f'stage_{stage}_diameter_ratio', 1.0)
            stage_thickness = thickness * design.get(f'stage_{stage}_thickness_ratio', 1.0)

            stage_mass = stage_length * np.pi * stage_diameter * stage_thickness * density
            stage_lengths.append(stage_length)
            stage_masses.append(stage_mass)
            total_length += stage_length

        # Adjust total length to match sum of stages
        length = total_length

        # Structural calculations
        structural_mass = sum(stage_masses)
        cross_section = np.pi * diameter * thickness
        I = (np.pi/64) * (diameter**4 - (diameter - 2*thickness)**4)  # Moment of inertia

        # Performance metrics
        performance = {
            'structural_mass': structural_mass,
            'total_mass': structural_mass + payload,
            'buckling_load': (np.pi**2 * youngs * I) / (length**2),
            'natural_freq': (1/(2*np.pi)) * np.sqrt((3 * youngs * I) / (structural_mass * length**3)),
            'stress_level': payload * g / cross_section,
            'safety_factor': (np.pi**2 * youngs * I) / (length**2 * payload * g),
            'thrust_to_weight': np.random.uniform(1.2, 2.0),
            'drag_coefficient': self.calculate_drag_coefficient(design),
            'dynamic_pressure': self.calculate_dynamic_pressure(design),
            'cost_estimate': self.estimate_cost(design, structural_mass),
            'delta_v': self.estimate_delta_v(design, structural_mass),
            'stage_masses': stage_masses,
            'stage_lengths': stage_lengths
        }

        return performance

    def calculate_drag_coefficient(self, design):
        """Simple drag coefficient estimation based on shape parameters"""
        base_cd = 0.3
        nosecone_effect = 0.1 * (1 - np.exp(-0.5*(design['nosecone_ratio'] - 1)))
        fin_effect = 0.05 * design['fin_count'] * design['fin_span']
        return base_cd + nosecone_effect + fin_effect

    def calculate_dynamic_pressure(self, design):
        """Estimate max dynamic pressure during ascent"""
        return 0.5 * 1.225 * (2000**2) * self.calculate_drag_coefficient(design)  # Simplified

    def estimate_cost(self, design, structural_mass):
        """Cost estimation model"""
        mat_cost = MATERIAL_DB[design['material']]['cost']
        base_cost = structural_mass * mat_cost
        complexity_factor = 1 + 0.1*design['fin_count'] + 0.05*design['stage_count']
        return base_cost * complexity_factor

    def estimate_delta_v(self, design, structural_mass):
        """Simplified delta-v estimation using Tsiolkovsky rocket equation"""
        isp = {'liquid': 350, 'solid': 280, 'hybrid': 310}[design['engine_type']]
        propellant_mass = structural_mass * design['propellant_mass_ratio'] / (1 - design['propellant_mass_ratio'])
        total_mass = structural_mass + design['payload_mass'] + propellant_mass
        dry_mass = structural_mass + design['payload_mass']
        return isp * g * np.log(total_mass / dry_mass)

    def evaluate_design(self, design, performance):
        """Enhanced design evaluation with more constraints"""
        mat_props = MATERIAL_DB[design['material']]

        constraints = {
            'mass_per_stage': all(m < MAX_MASS_PER_STAGE for m in performance['stage_masses']),
            'stress': performance['stress_level'] <= mat_props['max_stress'],
            'safety': performance['safety_factor'] >= MIN_SAFETY_FACTOR,
            'slenderness': design['length'] / design['diameter'] < 30,
            'thickness': design['wall_thickness'] / design['diameter'] > 0.005,
            'frequency': performance['natural_freq'] > 1.0,
            'dynamic_pressure': performance['dynamic_pressure'] < MAX_Q,
            'g_load': performance['thrust_to_weight'] < MAX_G_LOAD,
            'delta_v': performance['delta_v'] > 7500  # Minimum for LEO
        }

        # Calculate weighted score (could be enhanced with ML)
        weights = {
            'mass_per_stage': 1.5,
            'stress': 2.0,
            'safety': 2.5,
            'slenderness': 1.0,
            'thickness': 1.0,
            'frequency': 1.2,
            'dynamic_pressure': 1.3,
            'g_load': 1.1,
            'delta_v': 2.0
        }

        score = sum(weights[k] for k, v in constraints.items() if v)
        return score, constraints

    def train_surrogate_model(self, n_samples=500):
        """Train a machine learning model to predict design performance"""
        st.info("🚀 Training surrogate model for design optimization...")

        # Generate training data
        X = []
        y = []

        for _ in stqdm(range(n_samples)):
            design = self.generate_initial_population(1)[0]
            performance = self.calculate_performance(design)

            # Flatten design parameters
            x_vec = [
                design['diameter'],
                design['length'],
                design['wall_thickness'],
                list(MATERIAL_DB.keys()).index(design['material']),
                design['payload_mass'],
                design['stage_count'],
                design['nosecone_ratio'],
                design['fin_count'],
                design['fin_span'],
                ['liquid', 'solid', 'hybrid'].index(design['engine_type']),
                design['propellant_mass_ratio']
            ]

            # Add stage parameters
            for stage in range(design['stage_count']):
                x_vec.extend([
                    design.get(f'stage_{stage}_length_ratio', 0.33),
                    design.get(f'stage_{stage}_diameter_ratio', 1.0),
                    design.get(f'stage_{stage}_thickness_ratio', 1.0)
                ])

            # Pad with zeros for max stages (3)
            while len(x_vec) < 11 + 3*3:
                x_vec.append(0.0)

            # Target variables (performance metrics we care about)
            y_vec = [
                performance['structural_mass'],
                performance['total_mass'],
                performance['safety_factor'],
                performance['delta_v'],
                performance['cost_estimate']
            ]

            X.append(x_vec)
            y.append(y_vec)

        X = np.array(X)
        y = np.array(y)

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Train model
        self.surrogate_model = RandomForestRegressor(n_estimators=100)
        self.surrogate_model.fit(X_train, y_train)

        # Evaluate
        score = self.surrogate_model.score(X_test, y_test)
        st.success(f"Surrogate model trained with R² score: {score:.3f}")

    def optimize_design(self, initial_design, objectives):
        """Optimize design using surrogate model and constraints"""
        if not self.surrogate_model:
            self.train_surrogate_model()

        # Define optimization problem
        def objective(x):
            # Predict performance with surrogate model
            y_pred = self.surrogate_model.predict([x])[0]

            # Multi-objective optimization
            # Weights can be adjusted based on user preferences
            obj_value = 0
            if 'min_mass' in objectives:
                obj_value += 0.5 * y_pred[1]  # total_mass
            if 'max_safety' in objectives:
                obj_value -= 0.3 * y_pred[2]  # safety_factor
            if 'max_delta_v' in objectives:
                obj_value -= 0.4 * y_pred[3]  # delta_v
            if 'min_cost' in objectives:
                obj_value += 0.3 * y_pred[4]  # cost

            return obj_value

        # Constraints
        def constraint_safety(x):
            y_pred = self.surrogate_model.predict([x])[0]
            return y_pred[2] - MIN_SAFETY_FACTOR  # safety_factor > min

        def constraint_delta_v(x):
            y_pred = self.surrogate_model.predict([x])[0]
            return y_pred[3] - 7500  # delta_v > 7500

        constraints = [
            {'type': 'ineq', 'fun': constraint_safety},
            {'type': 'ineq', 'fun': constraint_delta_v}
        ]

        # Bounds (based on initial design)
        bounds = [
            (max(0.5, initial_design['diameter']*0.5), initial_design['diameter']*1.5),  # diameter
            (max(5, initial_design['length']*0.5), initial_design['length']*1.5),  # length
            (0.005, 0.15),  # wall_thickness
            (0, len(MATERIAL_DB)-1),  # material index
            (500, 25000),  # payload_mass
            (1, 3),  # stage_count
            (0.5, 3.0),  # nosecone_ratio
            (3, 6),  # fin_count
            (0.5, 2.0),  # fin_span
            (0, 2),  # engine_type index
            (0.7, 0.95)  # propellant_mass_ratio
        ]

        # Add bounds for stage parameters
        for stage in range(3):  # max stages
            bounds.extend([
                (0.1, 0.9),  # stage_length_ratio
                (0.7, 1.3),  # stage_diameter_ratio
                (0.7, 1.3)   # stage_thickness_ratio
            ])

        # Initial guess
        x0 = [
            initial_design['diameter'],
            initial_design['length'],
            initial_design['wall_thickness'],
            list(MATERIAL_DB.keys()).index(initial_design['material']),
            initial_design['payload_mass'],
            initial_design['stage_count'],
            initial_design['nosecone_ratio'],
            initial_design['fin_count'],
            initial_design['fin_span'],
            ['liquid', 'solid', 'hybrid'].index(initial_design['engine_type']),
            initial_design['propellant_mass_ratio']
        ]

        # Add stage parameters
        for stage in range(initial_design['stage_count']):
            x0.extend([
                initial_design.get(f'stage_{stage}_length_ratio', 0.33),
                initial_design.get(f'stage_{stage}_diameter_ratio', 1.0),
                initial_design.get(f'stage_{stage}_thickness_ratio', 1.0)
            ])

        # Pad with zeros for max stages (3)
        while len(x0) < 11 + 3*3:
            x0.append(0.0)

        # Run optimization
        result = minimize(
            objective,
            x0,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints,
            options={'maxiter': 50}
        )

        # Convert back to design dictionary
        optimized_design = self.vector_to_design(result.x)
        return optimized_design

    def vector_to_design(self, x):
        """Convert optimization vector back to design dictionary"""
        design = {
            'diameter': x[0],
            'length': x[1],
            'wall_thickness': x[2],
            'material': list(MATERIAL_DB.keys())[int(x[3])],
            'payload_mass': x[4],
            'stage_count': int(x[5]),
            'nosecone_ratio': x[6],
            'fin_count': int(x[7]),
            'fin_span': x[8],
            'engine_type': ['liquid', 'solid', 'hybrid'][int(x[9])],
            'propellant_mass_ratio': x[10]
        }

        # Add stage parameters
        stage_params_start = 11
        for stage in range(design['stage_count']):
            offset = stage * 3
            design[f'stage_{stage}_length_ratio'] = x[stage_params_start + offset]
            design[f'stage_{stage}_diameter_ratio'] = x[stage_params_start + offset + 1]
            design[f'stage_{stage}_thickness_ratio'] = x[stage_params_start + offset + 2]

        return design

    def visualize_design(self, design):
        """Create 3D visualization of the launch vehicle"""
        fig = go.Figure()

        # Colors for different materials
        material_colors = {
            'aluminum_2024': 'silver',
            'titanium_6Al-4V': 'darkgray',
            'carbon_composite': 'black',
            'inconel_718': 'goldenrod',
            'maraging_steel': 'steelblue'
        }

        color = material_colors.get(design['material'], 'blue')

        # Draw stages
        y_offset = 0
        for stage in range(design['stage_count']):
            stage_length = design['length'] * design.get(f'stage_{stage}_length_ratio', 0.33)
            stage_diameter = design['diameter'] * design.get(f'stage_{stage}_diameter_ratio', 1.0)

            # Cylinder for stage
            theta = np.linspace(0, 2*np.pi, 100)
            z = np.linspace(y_offset, y_offset + stage_length, 10)
            theta_grid, z_grid = np.meshgrid(theta, z)
            x_grid = (stage_diameter/2) * np.cos(theta_grid)
            y_grid = (stage_diameter/2) * np.sin(theta_grid)

            fig.add_trace(go.Surface(
                x=x_grid, y=y_grid, z=z_grid,
                colorscale=[[0, color], [1, color]],
                showscale=False,
                opacity=0.9,
                name=f'Stage {stage+1}'
            ))

            y_offset += stage_length

        # Draw nosecone - FIXED THE BROADCASTING ISSUE HERE
        nosecone_length = design['nosecone_ratio'] * design['diameter']
        z = np.linspace(y_offset, y_offset + nosecone_length, 50)
        r = (design['diameter']/2) * (1 - (z - y_offset)/nosecone_length)
        theta = np.linspace(0, 2*np.pi, 100)
        theta_grid, z_grid = np.meshgrid(theta, z)
        x_grid = r.reshape(-1, 1) * np.cos(theta_grid)  # Reshape r to (50,1) for broadcasting
        y_grid = r.reshape(-1, 1) * np.sin(theta_grid)  # Reshape r to (50,1) for broadcasting

        fig.add_trace(go.Surface(
            x=x_grid, y=y_grid, z=z_grid,
            colorscale=[[0, color], [1, color]],
            showscale=False,
            opacity=0.9,
            name='Nosecone'
        ))

        # Draw fins
        fin_span = design['fin_span']
        fin_count = design['fin_count']
        for i in range(fin_count):
            angle = 2*np.pi * i / fin_count
            fin_x = [0, fin_span * np.cos(angle), fin_span * np.cos(angle)]
            fin_y = [0, fin_span * np.sin(angle), fin_span * np.sin(angle)]
            fin_z = [0, 0, fin_span * 0.5]  # Fin height

            fig.add_trace(go.Mesh3d(
                x=fin_x, y=fin_y, z=fin_z,
                color='darkred',
                opacity=0.7,
                name=f'Fin {i+1}'
            ))

        fig.update_layout(
            title='Launch Vehicle Design',
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Height',
                aspectmode='manual',
                aspectratio=dict(x=1, y=1, z=3)
            ),
            height=800
        )

        return fig

    def plot_pareto_front(self):
        """Plot Pareto front of designs in the history"""
        if len(self.design_history) < 2:
            return None

        # Extract objectives
        costs = []
        masses = []
        delta_vs = []

        for design, perf in zip(self.design_history, self.performance_history):
            costs.append(perf['cost_estimate'] / 1e6)  # in millions
            masses.append(perf['total_mass'] / 1000)  # in tons
            delta_vs.append(perf['delta_v'] / 1000)  # in km/s

        fig = px.scatter_3d(
            x=costs, y=masses, z=delta_vs,
            labels={'x': 'Cost ($M)', 'y': 'Total Mass (tons)', 'z': 'Delta-V (km/s)'},
            title='Design Trade Space',
            opacity=0.7,
            size_max=10
        )

        fig.update_layout(
            scene=dict(
                xaxis_title='Cost ($M)',
                yaxis_title='Total Mass (tons)',
                zaxis_title='Delta-V (km/s)'
            ),
            height=700
        )

        return fig

def main():
    st.set_page_config(layout="wide", page_title="🚀 Advanced AI Launch Vehicle Designer")
    st.title("🚀 Advanced AI-Powered Launch Vehicle Design Optimizer")

    designer = LaunchVehicleDesigner()

    # Sidebar controls
    with st.sidebar:
        st.header("Design Parameters")
        payload_mass = st.slider("Payload mass (kg)", 1000, 20000, 5000)
        target_orbit = st.selectbox("Target orbit", ["LEO (200-2000km)", "GTO", "Lunar", "Mars"])
        budget = st.slider("Budget ($M)", 10, 500, 100)
        optimization_mode = st.selectbox("Optimization Mode",
                                      ["Balanced", "Minimize Mass", "Maximize Safety",
                                       "Maximize Delta-V", "Minimize Cost"])

        material_options = list(MATERIAL_DB.keys())
        default_material = material_options.index('titanium_6Al-4V')
        material = st.selectbox("Primary Material", material_options, index=default_material)

        st.header("Advanced Options")
        use_surrogate = st.checkbox("Use ML Surrogate Model", value=True)
        n_designs = st.slider("Number of initial designs", 1, 100, 20)
        optimize_designs = st.checkbox("Run optimization", value=True)

    # Main content
    tab1, tab2, tab3, tab4 = st.tabs(["Design Explorer", "3D Visualization", "Performance Analysis", "Trade Space"])

    if st.sidebar.button("Generate and Optimize Designs"):
        with st.spinner("Generating and evaluating designs..."):
            with tab1:
                st.header("Optimal Design Parameters")

                # Generate initial population
                designs = designer.generate_initial_population(n_designs)

                # Evaluate all designs
                best_score = -1
                best_design = None
                best_performance = None

                for design in stqdm(designs):
                    # Set payload mass from UI
                    design['payload_mass'] = payload_mass
                    design['material'] = material

                    performance = designer.calculate_performance(design)
                    score, constraints = designer.evaluate_design(design, performance)

                    # Store for Pareto analysis
                    designer.design_history.append(design)
                    designer.performance_history.append(performance)

                    if score > best_score:
                        best_score = score
                        best_design = design
                        best_performance = performance

                # Optimization
                if optimize_designs and use_surrogate:
                    objectives = []
                    if optimization_mode == "Minimize Mass":
                        objectives = ['min_mass']
                    elif optimization_mode == "Maximize Safety":
                        objectives = ['max_safety']
                    elif optimization_mode == "Maximize Delta-V":
                        objectives = ['max_delta_v']
                    elif optimization_mode == "Minimize Cost":
                        objectives = ['min_cost']
                    else:  # Balanced
                        objectives = ['min_mass', 'max_safety', 'max_delta_v', 'min_cost']

                    optimized_design = designer.optimize_design(best_design, objectives)
                    optimized_performance = designer.calculate_performance(optimized_design)

                    # Add to history
                    designer.design_history.append(optimized_design)
                    designer.performance_history.append(optimized_performance)

                    best_design = optimized_design
                    best_performance = optimized_performance

                if best_design:
                    st.success("✅ Optimal design generated!")

                    # Display design parameters
                    cols = st.columns(3)
                    with cols[0]:
                        st.subheader("Global Parameters")
                        st.metric("Total Length", f"{best_design['length']:.2f} m")
                        st.metric("Diameter", f"{best_design['diameter']:.2f} m")
                        st.metric("Wall Thickness", f"{best_design['wall_thickness']:.4f} m")
                        st.metric("Material", best_design['material'])
                        st.metric("Engine Type", best_design['engine_type'])

                    with cols[1]:
                        st.subheader("Stage Configuration")
                        for stage in range(best_design['stage_count']):
                            st.markdown(f"**Stage {stage+1}**")
                            st.metric("Length",
                                     f"{best_design['length'] * best_design.get(f'stage_{stage}_length_ratio', 0.33):.2f} m",
                                     help=f"{best_design.get(f'stage_{stage}_length_ratio', 0.33):.2f} of total length")
                            st.metric("Diameter",
                                     f"{best_design['diameter'] * best_design.get(f'stage_{stage}_diameter_ratio', 1.0):.2f} m")
                            st.metric("Mass", f"{best_performance['stage_masses'][stage]/1000:.2f} tons")

                    with cols[2]:
                        st.subheader("Aerodynamics")
                        st.metric("Nosecone Ratio", f"{best_design['nosecone_ratio']:.2f}")
                        st.metric("Fin Count", best_design['fin_count'])
                        st.metric("Fin Span", f"{best_design['fin_span']:.2f} m")
                        st.metric("Drag Coefficient", f"{best_performance['drag_coefficient']:.3f}")
                        st.metric("Max Q", f"{best_performance['dynamic_pressure']/1000:.1f} kPa")

                with tab2:
                    if best_design:
                        st.header("3D Design Visualization")
                        fig = designer.visualize_design(best_design)
                        st.plotly_chart(fig, use_container_width=True)

                with tab3:
                    if best_performance:
                        st.header("Performance Metrics")

                        cols = st.columns(2)
                        with cols[0]:
                            st.subheader("Structural Performance")
                            st.metric("Total Mass", f"{best_performance['total_mass']/1000:.2f} tons")
                            st.metric("Structural Mass", f"{best_performance['structural_mass']/1000:.2f} tons")
                            st.metric("Safety Factor", f"{best_performance['safety_factor']:.2f}")
                            st.metric("Natural Frequency", f"{best_performance['natural_freq']:.2f} Hz")
                            st.metric("Buckling Load", f"{best_performance['buckling_load']/1e6:.2f} MN")

                        with cols[1]:
                            st.subheader("Mission Performance")
                            st.metric("Delta-V", f"{best_performance['delta_v']/1000:.2f} km/s")
                            st.metric("Thrust-to-Weight", f"{best_performance['thrust_to_weight']:.2f}")
                            st.metric("Estimated Cost", f"${best_performance['cost_estimate']/1e6:.2f} M")
                            st.metric("Max Stress", f"{best_performance['stress_level']/1e6:.2f} MPa")
                            st.metric("Propellant Mass Ratio", f"{best_design['propellant_mass_ratio']:.3f}")

                with tab4:
                    st.header("Design Trade Space Analysis")
                    fig = designer.plot_pareto_front()
                    if fig:
                        st.plotly_chart(fig, use_container_width=True)
                    else:
                        st.warning("Need at least 2 designs to show trade space")

if __name__ == "__main__":
    main()


Writing app.py


In [ ]:
!pkill ngrok


In [ ]:


# 2. Start the Streamlit app in background
!nohup streamlit run app.py --server.port 8501 --server.headless true --server.enableCORS false &

# 3. Create an ngrok tunnel to the Streamlit app
from pyngrok import ngrok

# 4. (Optional) Set your ngrok auth token if needed
ngrok.set_auth_token("2v1WPxUy5A88vovYO0vTqkiwtf6_4dDm4btDEeocT8S9ZaTBU")

# 5. Connect the tunnel
public_url = ngrok.connect(8501)

# 6. Show the public URL
print(f"🚀 Streamlit app is live at: {public_url}")


nohup: appending output to 'nohup.out'
🚀 Streamlit app is live at: NgrokTunnel: "https://8207-34-133-2-21.ngrok-free.app" -> "http://localhost:8501"


NAME:
  config - update or migrate ngrok's configuration file

USAGE:
  ngrok config [flags]

DESCRIPTION: 
  The config command gives a quick way to create or update ngrok's configuration
  file. Use 'add-authtoken' or 'add-api-key' to set the corresponding properties.

  Use 'check' to test a configuration file for validity. If you have an old
  configuration file, you can also use 'upgrade' to automatically migrate to the
  latest version.

COMMANDS:
  add-api-key                    save api key to configuration file
  add-authtoken                  save authtoken to configuration file
  add-connect-url                adds the connect URL (connect_url) to configuration file for custom agent ingress
  add-server-addr                alias of add-connect-url
  check                          check configuration file
  edit                           edit configuration file
  upgrade                        auto-upgrade configuration file

OPTIONS:
      --config strings   path to config f